In [3]:
import numpy as np
from pymoo.indicators.hv import HV
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from itertools import product
from scipy.stats import qmc  # For Latin Hypercube Sampling
import torch
import gpytorch
import random
from PIL import Image
from datetime import datetime
#import scienceplots
from scipy.stats import norm
import functionBank as func
from sklearn.preprocessing import MinMaxScaler

# Input: Objective values for each solution
X = np.array([[1, 2], [2, 1], [3, 3], [4, 4], [1.5, 1.5]])  # Example input data
r = np.array([5, 5])  # Reference point for hypervolume calculation

# Step 1: Compute Pareto shells

def compute_pareto_shells(X):
    remaining = X.copy()
    shells = []

    while len(remaining) > 0:
        # Compute non-dominated front
        nds = NonDominatedSorting().do(remaining, only_non_dominated_front=True)
        pareto_front = remaining[nds]
        shells.append(pareto_front)

        # Remove selected Pareto front from remaining points
        remaining = np.delete(remaining, nds, axis=0)

    return shells

pareto_shells = compute_pareto_shells(X)

# Step 2: Compute hypervolume indicator

def compute_hypervolume(X, ref_point):
    hv = HV(ref_point)
    return hv.do(X)

# Step 3: Compute hypervolume improvement

def hypervolume_improvement(x, X, ref_point):
    # Find the first Pareto shell that does not dominate x
    for shell in pareto_shells:
        if not np.any(np.all(shell <= x, axis=1)):
            pareto_k = shell
            break

    # Compute hypervolume with x added to the shell
    hv_before = compute_hypervolume(pareto_k, ref_point)
    hv_after = compute_hypervolume(np.vstack([pareto_k, x]), ref_point)

    return hv_after - hv_before

# Example usage
x_new = np.array([2.5, 2.5])  # New candidate point
hv_improvement = hypervolume_improvement(x_new, X, r)

# Print outputs
print("Pareto Shells:", pareto_shells)
print("Hypervolume Improvement for x_new:", hv_improvement)


Pareto Shells: [array([[1. , 2. ],
       [2. , 1. ],
       [1.5, 1.5]]), array([[3., 3.]]), array([[4., 4.]])]
Hypervolume Improvement for x_new: 2.25


In [6]:
# Input: Objective values for each solution
X = np.array([[1, 2], [2, 1], [3, 3], [4, 4], [1.5, 1.5]])  # Example input data
r = np.array([5, 5])  # Reference point for hypervolume calculation

# Step 1: Compute Pareto shells
def compute_pareto_shells(X):
    remaining = X.copy()
    shells = []

    while len(remaining) > 0:
        # Compute non-dominated front
        nds = NonDominatedSorting().do(remaining, only_non_dominated_front=True)
        pareto_front = remaining[nds]
        shells.append(pareto_front)

        # Remove selected Pareto front from remaining points
        remaining = np.delete(remaining, nds, axis=0)

    return shells

pareto_shells = compute_pareto_shells(X)

# Step 2: Compute hypervolume indicator
def compute_hypervolume(X, ref_point):
    hv = HV(ref_point)
    return hv.do(X)

# Step 3: Compute hypervolume improvement
def hypervolume_improvement(x, X, ref_point):
    # Find the first Pareto shell that does not dominate x
    pareto_k = None
    for shell in pareto_shells:
        if not np.any(np.all(shell <= x, axis=1)):
            pareto_k = shell
            break

    # Check if pareto_k was assigned; if not, default to the last shell
    if pareto_k is None:
        pareto_k = pareto_shells[-1]

    # Compute hypervolume with x added to the shell
    hv_before = compute_hypervolume(pareto_k, ref_point)
    hv_after = compute_hypervolume(np.vstack([pareto_k, x]), ref_point)

    return hv_after - hv_before

# Compute hypervolume improvement for each point in the dataset D
hv_improvements = []
for x in X:
    hv_improvements.append(hypervolume_improvement(x, X, r))

# Print outputs
print("Pareto Shells:", pareto_shells)
print("Hypervolume Improvements for all points:", hv_improvements)


Pareto Shells: [array([[1. , 2. ],
       [2. , 1. ],
       [1.5, 1.5]]), array([[3., 3.]]), array([[4., 4.]])]
Hypervolume Improvements for all points: [8.0, 8.0, 3.0, 0.0, 8.25]


In [ ]:
#compute pareto shells
def compute_pareto_shells(X):
    remaining = X.copy()
    shells = []

    while len(remaining) > 0:
        # Compute non-dominated front
        nds = NonDominatedSorting().do(remaining, only_non_dominated_front=True)
        pareto_front = remaining[nds]
        shells.append(pareto_front)

        # Remove selected Pareto front from remaining points
        remaining = np.delete(remaining, nds, axis=0)

    return shells